In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/machine_translation/
!ls

/content/drive/MyDrive/machine_translation
bible_verses.csv				MT-Preparation
compute-bleu.py					nagamese.csv
compute-bleu.py.1				nagamese.csv-filtered.ng
config.yaml					nagamese.csv-filtered.ng.subword
english.csv					nagamese.csv-filtered.ng.subword.dev
english.csv-filtered.en				nagamese.csv-filtered.ng.subword.test
english.csv-filtered.en.subword			nagamese.csv-filtered.ng.subword.test.desubword
english.csv-filtered.en.subword.dev		nagamese.csv-filtered.ng.subword.train
english.csv-filtered.en.subword.test		run
english.csv-filtered.en.subword.test.desubword	source.model
english.csv-filtered.en.subword.train		source.vocab
en.translated					target.model
en.translated.desubword				target.vocab
models						train.log


In [ ]:
!rm -rf /content/drive/MyDrive/machine_translation/run/

In [ ]:
!pip3 install OpenNMT-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
%pwd
!ls

bible_verses.csv				MT-Preparation
compute-bleu.py					nagamese.csv
compute-bleu.py.1				nagamese.csv-filtered.ng
config.yaml					nagamese.csv-filtered.ng.subword
english.csv					nagamese.csv-filtered.ng.subword.dev
english.csv-filtered.en				nagamese.csv-filtered.ng.subword.test
english.csv-filtered.en.subword			nagamese.csv-filtered.ng.subword.test.desubword
english.csv-filtered.en.subword.dev		nagamese.csv-filtered.ng.subword.train
english.csv-filtered.en.subword.test		source.model
english.csv-filtered.en.subword.test.desubword	source.vocab
english.csv-filtered.en.subword.train		target.model
en.translated					target.vocab
en.translated.desubword				train.log
models


In [ ]:
config = '''# config.yaml

save_data: run

data:
    corpus_1:
        path_src: nagamese.csv-filtered.ng.subword.train
        path_tgt: english.csv-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: nagamese.csv-filtered.ng.subword.dev
        path_tgt: english.csv-filtered.en.subword.dev
        transforms: [filtertoolong]

src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

src_vocab_size: 6000
tgt_vocab_size: 6000

src_seq_length: 150
src_seq_length: 150

src_subword_model: source.model
tgt_subword_model: target.model

log_file: train.log
save_model: models/model.fren

early_stopping: 4

# Save a model checkpoint for each n steps
save_checkpoint_steps: 500  # More frequent checkpoints due to low dataset size

# Limit checkpoints to last n
keep_checkpoint: 3

seed: 2425

# Adjusted training parameters for better efficiency
train_steps: 4000        # Extended for deeper learning over limited dataset
valid_steps: 500          # More frequent validation for better tracking
warmup_steps: 500         # Reduced warm-up as dataset is limited
report_every: 100

world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 8192
num_workers: 0
batch_type: "tokens"
batch_size: 2048        # Reduced to better handle resource limitations
valid_batch_size: 1024  # Reduced to manage validation memory usage
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 1         # Slightly reduced for stability on small dataset
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
self_attn_type: scaled-dot
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
    config_yaml.write(config)



In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-11-09 16:03:34,762 INFO] Counter vocab from -1 samples.
[2024-11-09 16:03:34,762 INFO] n_sample=-1: Build vocab on full datasets.
[2024-11-09 16:03:35,765 INFO] Counters src: 2157
[2024-11-09 16:03:35,765 INFO] Counters tgt: 4016


In [ ]:
!ls

bible_verses.csv				MT-Preparation
compute-bleu.py					nagamese.csv
compute-bleu.py.1				nagamese.csv-filtered.ng
config.yaml					nagamese.csv-filtered.ng.subword
english.csv					nagamese.csv-filtered.ng.subword.dev
english.csv-filtered.en				nagamese.csv-filtered.ng.subword.test
english.csv-filtered.en.subword			nagamese.csv-filtered.ng.subword.test.desubword
english.csv-filtered.en.subword.dev		nagamese.csv-filtered.ng.subword.train
english.csv-filtered.en.subword.test		run
english.csv-filtered.en.subword.test.desubword	source.model
english.csv-filtered.en.subword.train		source.vocab
en.translated					target.model
en.translated.desubword				target.vocab
models						train.log
